In [1]:
from dotenv import load_dotenv
# API KEY 정보 불러오기
load_dotenv()

True

### PDF를 로드

In [2]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드
loader = PyPDFLoader("./data/소나기 - 황순원.pdf")
document = loader.load()
print(document[0].page_content[:200])

- 1 -소나기
황순원
소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . 
소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 
못하기나 한 듯이.
벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에
서 하더니 , 오늘은 징검다리 한가운


### 문서 분할

In [3]:
from langchain.text_splitter import CharacterTextSplitter

# 스플리터 지정
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",  # 분할기준
    chunk_size=3000,   # 사이즈
    chunk_overlap=500, # 중첩 사이즈
)
# 분할 실행
split_docs = text_splitter.split_documents(document)
# 총 분할된 도큐먼트 수
print(len(split_docs))

7


### 분할된 각 문서에 대한 요약 실행

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

# Map 단계에서 처리할 프롬프트 정의
# 분할된 문서에 적용할 프롬프트 내용을 기입합니다.
# 여기서 {pages} 변수에는 분할된 문서가 차례대로 대입되니다.
map_template = """다음은 문서 중 일부 내용입니다
{pages}
이 문서 목록을 기반으로 주요 내용을 요약해 주세요.
답변:"""

# Map 프롬프트 완성
map_prompt = PromptTemplate.from_template(map_template)

# Map에서 수행할 LLMChain 정의
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

C:\Users\turbo\AppData\Local\Temp\ipykernel_1672\3842122859.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
C:\Users\turbo\AppData\Local\Temp\ipykernel_1672\3842122859.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  map_chain = LLMChain(llm=llm, prompt=map_prompt)


### 각 문서의 요약본에 대한 통합

In [6]:
# Reduce 단계에서 처리할 프롬프트 정의
reduce_template = """다음은 요약의 집합입니다:
{doc_summaries}
이것들을 바탕으로 통합된 요약을 만들어 주세요.
답변:"""

# Reduce 프롬프트 완성
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [7]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain

# 문서의 목록을 받아들여, 이를 단일 문자열로 결합하고, 이를 LLMChain에 전달합니다.
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,                
    document_variable_name="doc_summaries" # Reduce 프롬프트에 대입되는 변수
)

# Map 문서를 통합하고 순차적으로 Reduce합니다.
reduce_documents_chain = ReduceDocumentsChain(
    # 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 `StuffDocumentsChain`의 컨텍스트를 초과하는 경우
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 때의 토큰 최대 개수입니다.
    token_max=4000,
)

C:\Users\turbo\AppData\Local\Temp\ipykernel_1672\4154098370.py:5: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(


### 통합체인(Combined Chain) 생성

In [8]:
from langchain.chains import MapReduceDocumentsChain

# 문서들에 체인을 매핑하여 결합하고, 그 다음 결과들을 결합합니다.
map_reduce_chain = MapReduceDocumentsChain(
    # Map 체인
    llm_chain=map_chain,
    # Reduce 체인
    reduce_documents_chain=reduce_documents_chain,
    # 문서를 넣을 llm_chain의 변수 이름(map_template 에 정의된 변수명)
    document_variable_name="pages",
    # 출력에서 매핑 단계의 결과를 반환합니다.
    return_intermediate_steps=False,
)

In [9]:
# Map-Reduce 체인 실행
# 입력: 분할된 도큐먼트(②의 결과물)
result = map_reduce_chain.invoke(split_docs)
# 요약결과 출력
print(result['output_text'])

이 문서는 황순원의 소설 "소나기"의 주요 내용을 요약한 것입니다. 이야기는 소년과 소녀의 짧고 순수한 만남과 그들 사이의 감정 변화를 중심으로 전개됩니다.

**등장인물**: 소년과 소녀

**배경**: 개울가, 징검다리, 논과 밭, 산, 수숫단 속, 소녀의 집

**줄거리**:
1. **첫 만남**: 소년은 개울가에서 물장난을 하던 소녀를 처음 만납니다. 소녀는 소년에게 하얀 조약돌을 던지며 "이 바보"라고 말하고 사라집니다. 소년은 소녀가 보이지 않는 날이 계속될수록 허전함을 느낍니다.
   
2. **함께하는 시간**: 소년과 소녀는 개울가에서 다시 만나 대화를 나누고, 소녀의 제안으로 산 너머로 가보며 논과 밭을 지나 허수아비를 흔들고, 다양한 꽃들을 꺾어 서로에게 보여줍니다. 소년은 소녀와 함께 있는 시간을 즐기며 내적 갈등을 겪습니다.

3. **소나기**: 소년과 소녀는 비를 피하기 위해 원두막으로 향하지만, 비를 완전히 막아주지 못하자 소년은 소녀를 수숫단 속으로 안내합니다. 소년은 소녀를 보호하기 위해 비를 맞으며 밖에 서 있습니다.

4. **소녀의 병**: 소녀는 그날 소나기를 맞고 병이 나서 앓게 됩니다. 소년은 소녀를 걱정하며 매일 개울가를 찾아가지만 소녀를 볼 수 없습니다. 어느 날 소녀는 병이 다 낫지 않았지만 답답해서 나왔다고 말하며 소년과 재회합니다.

5. **이별**: 소녀는 소년에게 대추를 주며 집을 곧 떠나게 될 것이라고 말합니다. 소년은 소녀와의 이별을 앞두고 아쉬움을 느낍니다. 소녀의 가족은 서울에서 사업에 실패하고 고향으로 돌아왔으며, 이제는 고향 집마저 남의 손에 넘기게 되었습니다. 소녀는 이사 가는 것을 싫어하지만 어른들의 결정이라 어쩔 수 없다고 말합니다.

6. **소년의 애정**: 소년은 밤에 몰래 덕쇠 할아버지네 호두밭에 가서 호두를 따서 소녀에게 주려고 합니다. 그는 소녀가 이사 가기 전에 개울가로 나와 달라는 말을 하지 못한 것을 후회합니다. 소년의 아버지는 윤 초시 댁에 제삿상에 놓을 닭을 가져가려고

## 독서감상문 작성하기

In [10]:
# 질문 템플릿 형식 정의
template = """다음은 소설에 대한 요약본입니다. 
다음의 내용을 독서 감상문 형식으로 작성해 주세요. 

독서 감상문의 형식은 다음과 같습니다:

처음: 글을 읽게 된 동기나 책을 처음 대했을 때의 느낌을 쓰고, 글의 종류나 지은이 소개, 주인
공이나 주제의 소개
중간: 주인공의 행동과 나의 행동을 비교해 보기도 하고, 글의 내용을 평가해 보기도 하며, 글
속에서 발견한 주제나 의미가 우리 사회에 어떻게 작용할 것인가를 씁니다. 그리고 글을 읽으면서 받은
감동을 쓰기도 합니다.
끝: 글의 내용을 정리하며, 교훈을 적어두기도 한다. 그리고 끝글은 지루하지 않도록 산뜻하게

{text}

답변:
"""

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=['text'])

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

output = llm_chain.run(text=result)
print(output)

C:\Users\turbo\AppData\Local\Temp\ipykernel_24624\3430557956.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  output = llm_chain.run(text=result)


### 독서 감상문: 황순원의 "소나기"

#### 처음
황순원의 소설 "소나기"는 한국 문학의 대표적인 단편 소설로, 순수하고 아름다운 첫사랑의 이야기를 담고 있습니다. 이 소설을 읽게 된 동기는 어릴 적 교과서에서 접했던 기억이 떠올라 다시 한 번 그 감동을 느끼고 싶었기 때문입니다. 소설의 주인공은 소년과 소녀로, 그들의 짧지만 깊은 만남과 이별을 중심으로 이야기가 전개됩니다.

#### 중간
소설의 첫 장면에서 소년은 개울가에서 물장난을 하는 소녀를 처음 만나게 됩니다. 소녀는 소년에게 하얀 조약돌을 던지며 "이 바보"라고 말하는데, 이 장면은 두 사람의 순수한 첫 만남을 상징합니다. 이후 소년과 소녀는 개울가와 논밭, 산마루 등 자연 속에서 함께 시간을 보내며 허수아비를 흔들고, 참외와 무우를 먹고, 꽃을 꺾으며 즐거운 시간을 보냅니다.

이 과정에서 소년과 소녀의 행동은 매우 순수하고 천진난만합니다. 소년은 소녀를 위해 자신의 저고리를 벗어주고, 소녀는 소년에게 대추를 건네며 이사 갈 것을 알립니다. 이러한 행동들은 두 사람의 순수한 우정과 사랑을 잘 보여줍니다. 특히 소나기를 맞아 원두막으로 피신하는 장면에서는 소년의 소녀에 대한 배려와 사랑이 잘 드러납니다.

이 소설은 단순한 첫사랑의 이야기를 넘어서, 자연 속에서의 순수한 교감을 중심으로 전개됩니다. 또한, 소녀의 가정이 경제적으로 어려움을 겪고 있다는 배경이 이야기의 깊이를 더합니다. 소녀의 병과 이사, 그리고 결국 소녀의 죽음은 소년에게 큰 슬픔을 안겨주며, 독자에게도 깊은 감동을 줍니다.

#### 끝
"소나기"는 소년과 소녀의 짧은 만남과 이별을 통해 순수한 사랑의 아름다움과 아쉬움을 잘 그려낸 작품입니다. 소설을 읽으며 느낀 감동은 순수한 사랑의 소중함과 그리움이었습니다. 소년과 소녀의 순수한 마음은 우리 사회에서도 여전히 중요한 가치로 남아있어야 할 것입니다.

이 소설을 통해 우리는 순수한 사랑의 아름다움과 그리움을 다시 한 번 되새길 수 있었습니다. 소년과 소녀의 이야기는 우리에